# Imports

In [72]:
# imports
# Selenium Wire
from seleniumwire import webdriver
from seleniumwire.utils import decode

# Selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait

# Else
from getpass import getpass
import time
import json

# Classes and functions

In [73]:
# dictionary of Accounts
accounts = {}

# Post class
class Post:
    def __init__(self, likes, date_of_pub):
        self.likes = likes
        self.date_of_pub = date_of_pub

# Account class
class Account:
    def __init__(self, followers, link):
        self.posts = []  # A list to store Post objects
        self.followers = followers
        self.link = link
        
    def append_post(self, post):
        if isinstance(post, Post):
            self.posts.append(post)
        else:
            raise ValueError("Only Post objects can be added to 'posts' attribute.")

## Set up for scraping

In [74]:
# setting up webdriver
driver = webdriver.Chrome()


# open the webpage
driver.get("https://www.instagram.com")

### Logging in

In [75]:
# target username and password
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
    "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
    "input[name='password']")))

# enter username and password
username.clear()
username.send_keys(input("Enter you username: "))
password.clear()
password.send_keys(getpass("Enter your password: "))

# target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
    "button[type='submit']"))).click()

# We are logged in! test_username: ninamuray3 test_password: wexna4-suxkeb-gyFwov

Enter you username: ninamuray3
Enter your password: ········


### NOT NOW handling

In [76]:
# hadle NOT NOW. Comment out one of them if not needed or gives TimeOut Exception
# 1) Save details
not_now = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, 
    '//div[contains(text(), "Not now")]'))).click()

# 2) Notifications
not_now2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, 
    '//button[contains(text(), "Not Now")]'))).click()

### Search hashtags

In [77]:
keyword = input("Enter the hashtag without '#': ")
driver.get("https://www.instagram.com/explore/tags/" + keyword + "/")
 
# Wait for 5 seconds
time.sleep(5)

Enter the hashtag without '#': sheffieldcoffee


### Start Scarping

In [78]:
# find and click on the first post to activate the "Next" button
first_post = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, 
    "//a[contains(@href, '/p/')]"))).click()

In [81]:
action = ActionChains(driver)

# hover over username
element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div._aasi > div > header > div._aaqy._aaqz > div._aar0._ad95._aar1 > div.x78zum5 > div > div > span > div > div > a')))
action.move_to_element(element).perform()

# catch response from https://www.instagram.com/api/v1/users/{user_id}/info
response = driver.wait_for_request('https://www.instagram.com/api/v1/users/', 10).response
response_decoded = json.loads(decode(response.body, response.headers.get('Content-Encoding', 'identity')))

# record the account and/or post if the account is buisness
if response_decoded["user"]["is_business"]:
    username = response_decoded["user"]["username"]
    link = element.get_attribute('href')
    followers = response_decoded["user"]["follower_count"]
    likes = driver.find_element(By.CSS_SELECTOR, 'div._ae2s._ae3v._ae3w > section._ae5m._ae5n._ae5o > div > div > span > a > span > span').text
    date_of_pub = driver.find_element(By.CSS_SELECTOR, 'div._ae5u._ae5v._ae5w > div > div > a > span > time').get_attribute('datetime')

    if username in accounts:
        accounts[username].append_post(Post(likes, date_of_pub))
    else:
        profile = Account(followers, link)
        profile.append_post(Post(likes, date_of_pub))
        accounts[username] = profile

print(accounts)
print(json.dumps(response_decoded, sort_keys = True, indent = 4))

{'samdai_official': <__main__.Account object at 0x7f90de2c0490>}
{
    "status": "ok",
    "user": {
        "account_badges": [],
        "account_category": "",
        "account_type": 2,
        "address_street": "",
        "ads_incentive_expiration_date": null,
        "ads_page_id": "109180117432668",
        "ads_page_name": "samdai_official",
        "auto_expand_chaining": null,
        "bio_links": [
            {
                "link_id": "17955677267529297",
                "link_type": "external",
                "lynx_url": "https://l.instagram.com/?u=https%3A%2F%2Fsamdai.co.uk%2F&e=AT2zr8bICcHX0aDhzCqvYRFlzjxvMUH9AhS5G1o_Gx1nK9dRda5mmyqlxjKkW80-D2RMrDGAQGC8CneznoDTuaGAia29VvwTcniW6CDMx6rugrdtecoIQOk",
                "open_external_url_with_in_app_browser": true,
                "title": "Samdai",
                "url": "https://samdai.co.uk"
            }
        ],
        "biography": "Same day fashion delivery - in 2-hour! \ud83d\ude9a\ud83d\udca8\nSamdai VIP - Disc